In [691]:
from scipy.sparse import *
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import datasets
from importlib import reload as rl
import aux

In [578]:
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

news_bunch = datasets.fetch_20newsgroups(subset='all',
                              remove=('headers', 'footers', 'quotes'))
 
X_news_all = TfidfVectorizer().fit_transform(news_bunch.data)

Y_news_all = news_bunch.target


#### Inicializa corpus e também a versão Tfidf dele

In [683]:
cp = [ 
       'o sapo nao lava op peh nao lava porque nao quer', 
       'ele mora la na lagoa não lava o pé porque não quer', 
       'mas que chulé!', 
       'Ele mora lá na lagoa não lava o pé porque não quer' 
       ]
mcp = TfidfVectorizer().fit_transform(cp)
mcopy = mcp.copy()
mc0 = mcp[0]
mc0ar = np.array(mc0)
def corpus_small():
    global mcp, mcopy, mc0, mc0ar
    mcp = TfidfVectorizer().fit_transform(cp)
    mcopy = mcp.copy()
    mc0 = mcp[0]
    mc0ar = np.array(mc0)
def corpus_big():
    mcp = X_news_all
    mc0 = mcp[0]
    mc0ar = np.array(mc0)
    mcopy = mcp.copy()


In [543]:
def time_function(func, *args, **kargs):
#     print("func: ", func)
    if args and kargs:
        t0 = time()
        func(*args, kargs)
    elif args and not kargs:
        t0=time()
        func(*args)
    elif not args and kargs:
        t0 = time()
        func(kargs)
    else:
        t0 = time()
        func()
    return time() - t0

def loop_time_function(n, func, *args, **kargs):
    """" Executa n vezes a função func, passando args e kargs como argumento.
    Seu retorno é a soma dos tempos de execução de cada chamada à função.
    """
    return sum( (time_function(func, *args, **kargs) for _ in range(n) ) )

## Código para subtração manual, elemento a elemento
    + executou de forma consistentemente mais lenta que as versões vetorizadas

In [544]:
def manual_sub(cpy=mcopy, dest=mcp, idx = 0):
    global mc0
    mcp[0] - mc0
    mcp[1] - mc0
    mcp[2] - mc0
    mcp[3] - mc0

## Subtração ao se converter tudo para np.array foi a mais veloz
    
    Contudo, inviável para conjuntos maiores de dados (alta complexidade de espaço) conforme demonstrado ao se tentar utilizar essa estratégia com o dataset de notícias
    20newsgroup.
```python
# PROBLEMA: COMPLEXIDADE DE ESPAÇO
def numpy_array_sub(cpy=mcopy, dest=mcp, idx = 0):
    cpy.toarray() - cpy[idx].toarray()
```

In [804]:
# PROBLEMA: COMPLEXIDADE DE ESPAÇO
def numpy_array_sub(cpy=mcopy, dest=mcp, idx = 0):
    return cpy.toarray() - cpy[idx].toarray()


In [798]:
# Define algumas funções candidatas a substituir a subtração com conversão direta
# para array a partir de matriz esparsa.

# Ok.
def numpy_gambs_sub(cpy=mcopy, dest=mcp, idx = 0):
    return np.array([*mcp]) - np.array(mcp.shape[0]*[mcp[idx]])

def _sub(a,b):
    return a - b
_vsub = np.frompyfunc(sub, 2, 1)
# Ok, however in 'ramdom' order (correct submatrix)
def vectorized_sub(cpy=mcopy, dest=mcp, idx = 0):
    return _vsub(np.array([*mcp]), mc0)

# Ok. Same as 'vectorized_sub' 
def vectorized_gambs(cpy=mcopy, dest=mcp, idx = 0):
    return vsub([*mcp], mc0)

# TESTANTO
def sub_stupid(a, b):
    return a - b

stupid_vec = np.frompyfunc(sub_stupid, 2, 1)
def stupid_method(cpy=mcopy, dest=mcp, idx = 0):
    mc0ar = np.array(mc0)
    return stupid_vec([*mcp], mc0ar)

# IDEIA ORIGINALL
_to_sub = np.arange( 0, 4, dtype=csr_matrix ) 
_to_sub[np.array(range(mcp.shape[0]))] = mc0
def original_idea(cpy=mcopy, dest=mcp, idx = 0):
    return [*mcp] - _to_sub

# Parece LENTO
def original_lil(cpy=mcopy, dest=mcp, idx = 0):
    return [*mcp.tolil()] -np.array([mc0])
# Ok. Simples e FUNCIONA
def simple_sub(cpy=mcopy, dest=mcp, idx = 0):
    return np.array([*mcp]) - [mcp[0]]

In [785]:

# print(mcp)
mc_big()
try:
    t0 = time()
    mcp.toarray()
except BaseException:
    print(time() - t0)

In [628]:
# loop_time_function(1, stupid_method)
print(type(mcp[0]))
print(mcp[0])
print('ok')
print(type(mcp[0] - mc0))
print(mcp[1] - mc0)

<class 'scipy.sparse.csr.csr_matrix'>
  (0, 24810)	0.13884931659983243
  (0, 114159)	0.054736714669179454
  (0, 110739)	0.037974687974256474
  (0, 29449)	0.13122965511734616
  (0, 89588)	0.17036870960023223
  (0, 93532)	0.49835327837061044
  (0, 52374)	0.0866530013500092
  (0, 26465)	0.08914949300938678
  (0, 96770)	0.06489693974285242
  (0, 39025)	0.09292472333557171
  (0, 22362)	0.07108512930815475
  (0, 116790)	0.18121723459198072
  (0, 73447)	0.08082262581632703
  (0, 25769)	0.036492943561153894
  (0, 71770)	0.06356353510772615
  (0, 96012)	0.08438145186248155
  (0, 101388)	0.07788730502152184
  (0, 79249)	0.09799294739501875
  (0, 44272)	0.1981816470395562
  (0, 22945)	0.05826315190620052
  (0, 31039)	0.17878903113578784
  (0, 98219)	0.13122965511734616
  (0, 118091)	0.05049498528900672
  (0, 25260)	0.02107934232598577
  (0, 102231)	0.12606018701082408
  :	:
  (0, 65467)	0.0450039490867083
  (0, 95129)	0.09279827218104489
  (0, 32176)	0.12719876177804615
  (0, 108815)	0.0467852531

In [800]:
benchmark = {}

In [801]:
# Apenas para efeitos de benchmark, usar um corpus pequeno
mc_small()
# mc_big()

In [802]:
# nai = time_function(naive_sub)
rl(aux)
nn = 21
loop = 10
lista = [
    ('numpy_gambs_sub', numpy_gambs_sub),
    ('vectorized_sub', vectorized_sub),
    ('vectorized_gambs', vectorized_gambs),
    ('stupid_method', stupid_method),
    ('original_idea', original_idea),
    ('original_lil', original_lil),
    ('simple_sub', simple_sub),
]

In [803]:
for _ in range(loop):
    if not benchmark.get(nn):
      benchmark[nn] = {}
      for key, _ in lista:
        benchmark[nn][key] = []
    else:
      d = benchmark[nn]
      for key, fun in lista:
        if key not in d:
            benchmark[nn][key] = []

    dict_times = {}
    for key, func in lista:
      spent = loop_time_function(nn, func)
      dict_times[key] = spent
      benchmark[nn][key].append(spent)



    # dict_times['numpy_sub'] = loop_time_function(nn, numpy_array_sub)
    rl(aux)
    aux.ShowOrderedDict(dict_times, option='values')
    print('\n')

original_idea        => 0.0129402
simple_sub           => 0.0131907
stupid_method        => 0.0141993
vectorized_gambs     => 0.0160687
vectorized_sub       => 0.0175583
original_lil         => 0.0194480
numpy_gambs_sub      => 0.0237002


original_idea        => 0.0108893
vectorized_gambs     => 0.0111892
stupid_method        => 0.0113201
vectorized_sub       => 0.0114059
simple_sub           => 0.0127192
numpy_gambs_sub      => 0.0143750
original_lil         => 0.0188763


original_idea        => 0.0109982
vectorized_gambs     => 0.0111163
vectorized_sub       => 0.0111203
stupid_method        => 0.0112035
simple_sub           => 0.0127413
numpy_gambs_sub      => 0.0142477
original_lil         => 0.0180340


vectorized_sub       => 0.0134778
numpy_gambs_sub      => 0.0138149
original_idea        => 0.0138917
vectorized_gambs     => 0.0159562
stupid_method        => 0.0165126
simple_sub           => 0.0167024
original_lil         => 0.0199196


original_idea        => 0.0110190
vector

In [791]:
aux.ShowOrderedDictList(benchmark.get(nn, {}), option='values')

original             => 0.0139307
stupid               => 0.0143169
vec_gam              => 0.0143402
vec_sub              => 0.0143610
numpy_gam            => 0.0179420
lil                  => 0.0194874
